In [1]:
#set up

import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
from fredapi import Fred
from math import sqrt
import matplotlib.pyplot as plt

C:\Users\ianda\miniconda3\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
#creates a lagged column for each column

def buildLaggedFeatures(df, tau):
    df_lagged = df.copy()
    
    #creating a duplicate column for each existing column and shifting it by tau, thus creating "lag"
    
    for window in range(0, tau + 1):
        shifted = df.shift(window)
        shifted.columns = [x + "_tau" + str(window) for x in df.columns]

        df_lagged = pd.concat((df_lagged, shifted), axis=1)

    
    return df_lagged

In [3]:
# connecting to FRED API
fred = Fred(api_key='0e929d96cfbf2133c9a346a74670f033')

# gathering fred data (currently as series)
data1 = fred.get_series('PCEC96')            # Real personal Consumption expenditures

data2 = fred.get_series('CSCICP03USM665S')   # Consumer Opinion Surveys: Confidence Indicators:
                                             #    Composite Indicators
    
data3 = fred.get_series('BOGZ1FA154104023Q') # Households and Nonprofit Organizations; Debt 
                                             # Service Payments (FSIs), Transactions

In [4]:
d1 = pd.DataFrame({'date':data1.index, 'PCEC96':data1.values})
d2 = pd.DataFrame({'date':data2.index, 'CSCICP03USM665S':data2.values})
d3 = pd.DataFrame({'date':data3.index, 'BOGZ1FA154104023Q':data3.values})


In [5]:
d3 = d3.rename(columns={'BOGZ1FA154104023Q':'temp'})

In [6]:
# filling in NaN values of TDSP with previous values if null

d3 = pd.merge(d1, d3, how="left", on="date")

d3 = buildLaggedFeatures(d3, 3)

d3 = d3[['date', 'temp', 'temp_tau1', 'temp_tau2']]

d3["temp_tau1"] = d3["temp_tau1"].fillna(d3["temp_tau2"])

d3["temp"] = d3["temp"].fillna(d3["temp_tau1"])

d3 = d3[['date', 'temp']]


In [7]:
# MERGING!
# d1 is the oldest so we will merge on it (left merge)

df = pd.merge(d1, d2, how="left", on="date")
df = pd.merge(df, d3, how="left", on="date")

In [8]:
df = df.rename(columns={'PCEC96':'Consumption (B)'})
df = df.rename(columns={'temp':'Debt Service Payments (M)'})
df = df.rename(columns={'CSCICP03USM665S':'Consumer Conf (%)'})

In [9]:
# read in stimulus payment plans

d4 = pd.read_excel('stimulus_payments.xlsx')
d4 = d4.rename(columns={'Date':'date'})

In [10]:
# merge in stimulus payment plans
df = pd.merge(df, d4, how="left", on="date")

In [11]:
df['date'] = df.date.dt.to_period("M")

In [12]:
# drop everything before 2020-01 and after 2022-04
df = df[df['date'] >= '2020-01']


In [13]:
df = buildLaggedFeatures(df, 3)

In [14]:
df = df[['date', 'Consumption (B)', 'Consumer Conf (%)', 'Debt Service Payments (M)',
         'Stimulus Payement Total (Billions)',
         'Stimulus Payement Total (Billions)_tau1',
         'Stimulus Payement Total (Billions)_tau2',
         'Stimulus Payement Total (Billions)_tau3']]

df = df.rename(columns={'Stimulus Payement Total (Billions)':'Stimi'})
df = df.rename(columns={'Stimulus Payement Total (Billions)_tau1':'Stimi_lag1'})
df = df.rename(columns={'Stimulus Payement Total (Billions)_tau2':'Stimi_lag2'})
df = df.rename(columns={'Stimulus Payement Total (Billions)_tau3':'Stimi_lag3'})


In [15]:
df = df.rename(columns={'Consumption (B)':'Consumption'})
df = df.rename(columns={'Debt Service Payments (M)':'Debt'})

In [16]:
# drop everything after 2022-04
df = df[df['date'] < '2022-04']

In [17]:
# converting debts from 
df["Debt"] = df["Debt"] / 100.00

In [18]:
df

,date,Consumption,Consumer Conf (%),Debt,Stimi,Stimi_lag1,Stimi_lag2,Stimi_lag3
732,2020-01,13277.0,101.600997,16358.48,0.0,NaN,NaN,NaN
733,2020-02,13313.6,101.166637,16358.48,0.0,0.0,NaN,NaN
734,2020-03,12459.6,100.153904,16358.48,0.0,0.0,0.0,NaN
735,2020-04,10962.8,99.052402,16155.24,280.0,0.0,0.0,0.0
736,2020-05,11900.6,98.545005,16155.24,0.0,280.0,0.0,0.0
737,2020-06,12588.1,98.494485,16155.24,0.0,0.0,280.0,0.0
738,2020-07,12814.1,98.512033,16183.96,0.0,0.0,0.0,280.0
739,2020-08,12884.9,98.716813,16183.96,0.0,0.0,0.0,0.0
740,2020-09,13068.1,99.033829,16183.96,0.0,0.0,0.0,0.0
741,2020-10,13102.3,99.184744,16253.00,0.0,0.0,0.0,0.0


In [19]:
result_lag_0 = sm.ols(formula= "Consumption ~ Stimi", data=df).fit()
result_lag_1 = sm.ols(formula= "Consumption ~ Stimi_lag1", data=df).fit()
result_lag_2 = sm.ols(formula= "Consumption ~ Stimi_lag2", data=df).fit()
result_lag_3 = sm.ols(formula= "Consumption ~ Stimi_lag3", data=df).fit()

In [20]:
print(result_lag_0.summary())

                            OLS Regression Results                            
Dep. Variable:            Consumption   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     1.835
Date:                Mon, 14 Nov 2022   Prob (F-statistic):              0.188
Time:                        20:24:59   Log-Likelihood:                -215.17
No. Observations:                  27   AIC:                             434.3
Df Residuals:                      25   BIC:                             436.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.339e+04    148.810     89.947      0.0

In [21]:
print(result_lag_1.summary())

                            OLS Regression Results                            
Dep. Variable:            Consumption   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.035
Method:                 Least Squares   F-statistic:                    0.1458
Date:                Mon, 14 Nov 2022   Prob (F-statistic):              0.706
Time:                        19:16:18   Log-Likelihood:                -208.54
No. Observations:                  26   AIC:                             421.1
Df Residuals:                      24   BIC:                             423.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.334e+04    160.263     83.233      0.0

In [22]:
print(result_lag_2.summary())

                            OLS Regression Results                            
Dep. Variable:            Consumption   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.043
Method:                 Least Squares   F-statistic:                  0.007640
Date:                Mon, 14 Nov 2022   Prob (F-statistic):              0.931
Time:                        19:16:18   Log-Likelihood:                -201.08
No. Observations:                  25   AIC:                             406.2
Df Residuals:                      23   BIC:                             408.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.331e+04    167.891     79.295      0.0

In [23]:
print(result_lag_3.summary())

                            OLS Regression Results                            
Dep. Variable:            Consumption   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.035
Method:                 Least Squares   F-statistic:                    0.2269
Date:                Mon, 14 Nov 2022   Prob (F-statistic):              0.639
Time:                        19:16:18   Log-Likelihood:                -192.74
No. Observations:                  24   AIC:                             389.5
Df Residuals:                      22   BIC:                             391.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.332e+04    170.060     78.352      0.0

In [24]:
# df.to_excel(r'C:\Users\ianda\Documents\ECON501\finalData.xlsx', index = False)